In [5]:
from ibm_quantum_widgets import CircuitComposer
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *


Comenzamos almacenando en variables los diferentes métodos de ejecución del circuito para su posterior uso:

In [ ]:
backend_unitary = Aer.get_backend('unitary_simulator')
backend_qasm = Aer.get_backend('qasm_simulator')
backend_stvec = Aer.get_backend('statevector_simulator')

Ahora nos disponenmos a construir el cirucuito intoduciendo una puerta CNOT y otra de Hadamard

In [6]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.cx(qreg_q[0], qreg_q[1])
circuit.h(qreg_q[0])

In [7]:
circuit.draw()

┌───┐
q_0: ──■──┤ H ├
     ┌─┴─┐└───┘
q_1: ┤ X ├─────
     └───┘     
  c: ══════════

Veamos la matriz que interviene en nuestro circuito.

In [16]:
unitary= execute(circuit,backend_unitary,shots=1024).result().get_unitary()
array_to_latex(unitary)


<IPython.core.display.Latex object>

En este caso el estado inicial de nuestro circuito será $|00\rangle $. Luego el vector de estado final será:


In [17]:
final_state= execute(circuit,backend_stvec,shots=1024).result().get_statevector()
array_to_latex(final_state)

<IPython.core.display.Latex object>

In [18]:
circuit.barrier(qreg_q[0], qreg_q[1])
circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[0])